In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import re
from descartes import PolygonPatch
plotly.tools.set_credentials_file(username='pacoanes', api_key='5oW2wBsYr0Gv6euPzitK')
data = pd.read_csv('GSAF5.csv', engine='python')



Voy a realizar una breve limpieza para poder hacer un analisis rapido y poder plantearme una hipotesis. Para ello limpiare las fechas para obtener años y meses.

In [2]:
#miro la forma para ver de que esta compuesta
data.shape

(5992, 24)

In [3]:
#analizo los elementos nulos de cada columna
null_cols = data.isnull().sum()

In [4]:
#Elimino las columnas que no necesito
datan = data.drop(columns=['Unnamed: 22', 'Unnamed: 23', 'Investigator or Source','pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'Species ', 'Time'])

In [5]:
#Elimino los años que estan a cero
datani = datan[datan.Year != 0]

Vamos a analizar los ataques de los ultimos años.

In [6]:
datanis = datani[datani.Year > 1940]

In [7]:
datanis['Date'] = datanis['Date'].str.replace('-', ' ')
datanis['Date'] = datanis['Date'].str.replace('\d+', '')
datanis['Date'] = datanis['Date'].str.replace('Reported', '')
datanis['Date'] = datanis['Date'].str.replace('Early', '')
datanis['Date'] = datanis['Date'].str.replace('Late', '')
datanis['Date'] = datanis['Date'].str.replace(' ', '')
datanis['Date'] = datanis['Date'].str.replace('Summer, summer, summerof ', '')

/usr/local/Cellar/ipython/7.3.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/Cellar/ipython/7.3.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/Cellar/ipython/7.3.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [8]:
df = datanis[datanis.Date != ""]

In [9]:
df2 = df[df["Date"].isin(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','Dec'])]

Limpiaremos otras columnas mas sencillas como el sexo y la de ataques.

In [10]:
dasa = df2.drop(columns=['Case Number','Location','Name','Age'])

In [11]:
dasa=dasa.rename(columns = {'Sex ':'Sex'})
dasa=dasa.rename(columns = {'Fatal (Y/N)':'Fatal'})
dasa=dasa.rename(columns = {'Date':'Month'})

In [12]:
dasa1 = dasa[dasa["Sex"].isin(['M', 'F'])]

In [13]:
dasa1['Fatal'] = dasa1['Fatal'].str.replace(' N', 'N')

/usr/local/Cellar/ipython/7.3.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Ahora mismo ya tenemos un dataset minimo para poder realizar consulta y al que podemos volver mas tarde para volver a limpiar si queremos usar informacion como las actividades o el tipo de daño provocado

In [14]:
dasa1.head()

,Month,Year,Type,Country,Area,Activity,Sex,Injury,Fatal,original order
0,Sep,2016,Unprovoked,USA,Florida,Surfing,M,Minor injury to thigh,N,5993
1,Sep,2016,Unprovoked,USA,Florida,Surfing,M,Lacerations to hands,N,5992
2,Sep,2016,Unprovoked,USA,Florida,Surfing,M,Lacerations to lower leg,N,5991
3,Sep,2016,Unprovoked,AUSTRALIA,Victoria,Surfing,M,Struck by fin on chest & leg,N,5990
4,Sep,2016,Unprovoked,AUSTRALIA,Victoria,Surfing,M,No injury: Knocked off board by shark,N,5989


Convertimos el dataframe en un csv

In [15]:
dasa1.to_csv('data_limpia.csv')
dasa1.Fatal.value_counts()


N          3165
Y           707
UNKNOWN      20
Name: Fatal, dtype: int64

In [25]:
df = pd.DataFrame()
df["dead"] = dasa1[dasa1["Fatal"]=='Y'].groupby(["Country"]).count()["Fatal"]
df["accident"] = dasa1.groupby(["Country"]).count()["Fatal"]
df["frate"] = df["dead"]/ df["accident"]
df1 = df.reset_index().rename_axis(None).rename_axis(None, axis=1)
df1.head()

,Country,dead,accident,frate
0,PHILIPPINES,1,1,1.000000
1,TONGA,2,3,0.666667
2,AMERICAN SAMOA,3,3,1.000000
3,ANDAMAN / NICOBAR ISLANDAS,1,1,1.000000
4,ARUBA,1,1,1.000000


In [23]:
null_cols = df.isnull().sum()
null_cols

dead        0
accident    0
frate       0
dtype: int64